# PIC-SURE API Use-Case: Querying on Genomic Variables

This is a tutorial notebook, aimed to be quickly up and running with the R PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE R API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

As part of the BioData Catalyst initiative, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI). 

Original data exposed through PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By easing the process of data extraction, it allows investigators to focus on the downstream analyses and to facilitate reproducible science.

Both phenotypic and genomic variables are accessible through the PIC-SURE API.

### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.

The R/python PIC-SURE API is a small part of the entire PIC-SURE platform.

The R API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API R Library GitHub repos:
* https://github.com/hms-dbmi/pic-sure-r-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-r-client
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds



 -------

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisites
- R 3.4 or later

### Install packages

In [ ]:
source("R_lib/requirements.R")

#### Installing the latest PIC-SURE API library from GitHub

Installation of the two components of the PIC-SURE API from GitHub, that is the PIC-SURE adapter and the PIC-SURE Client.

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
install.packages("https://cran.r-project.org/src/contrib/Archive/devtools/devtools_1.13.6.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/R6_2.5.0.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/hash_2.2.6.1.tar.gz", repos=NULL, type="source")
install.packages(c("urltools"),repos = "http://cran.us.r-project.org")
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", force=T)

In [ ]:
library(stringr)
library(dplyr)

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- Resource ID
- User-specific security token


If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")

In [ ]:
connection <- picsure::connect(url = PICSURE_network_URL,
                               token = token)

In [ ]:
resource <- bdc::get.resource(connection,
                               resourceUUID = resource_id)

Two objects are created here: a `connection` and a `resource` object.

Since we will only be using a single resource, **the `resource` object is the only one we will need to proceed with the data analysis**.

It is connected to the specific resource we supplied and enables us to query and retrieve data from this database.

## Building the query with the PIC-SURE API

We are going to create a new query request from the PIC-SURE resource that was specified above.  For this example, we will limit the query to a single study, a single phenotype (gender and age range), and two genomic filters.

First, we will create a new query instance.

In [ ]:
my_query <- bdc::new.query(resource=resource)


#### Limiting the query to a single study

By default, new query objects are automatically populated with all the consent groups for which you are authorized to access.  For this example we are going to clear the existing consents and specify a single consent group that represents accessing only the NHLBI TOPMed: Study of African Americans, Asthma, Genes and Environment (SAGE) study.

In [ ]:
# Here we show all the studies that you have access to
bdc::query.show(bdc::new.query(resource = resource))

In [ ]:
# Here we delete that access and add only the consent code corresponding to the SAGE study
bdc::query.filter.delete(my_query, "\\_consents\\")

In [ ]:
# Here we show that we have only selected a single study
bdc::query.filter.add(query = my_query,
                      keys = "\\_consents\\",
                      as.list(c("phs000921.c2")))

*Note that trying to manually add a consent group which you are not authorized to access will result in errors downstream.*

#### List available phenotypic variables

Once a connection to the desired resource has been established, it is helpful to search for variables related to our search query. To this end, we will use the `dictionary` method of the `resource` object to create a data dictionary instance to search for variables.

In [ ]:
# search for variables that are part of the SAGE study
fullVariablesDict <- bdc::find.in.dictionary(resource, "") %>% bdc::extract.entries()

# extract the phenotypic variables
fullPhenotypeVars <- fullVariablesDict[fullVariablesDict$HpdsDataType == "phenotypes", ]

# display phenotypic vars for SAGE study
fullPhenotypeVars[stringr::str_detect(fullPhenotypeVars$name, fixed("(SAGE)")), c("name", "patientCount", "observationCount", "categorical", "min", "max", "HpdsDataType")]

#### Add categorical phenotypic variable (gender) to the query

A `dictionary` instance enables us to retrieve matching records by searching for a specific term. The `extract.entries` method can be used to retrieve information about all available variables. For instance, looking for variables containing the term `Sex of participant` is done this way:

In [ ]:
found_terms <- bdc::find.in.dictionary(resource = resource, 
                                        term = "Sex of participant")

We now will look for variables containing the term `Sex of participant` which also contain "`(SAGE)`" . This will allow us to find the specific name associated with our variable of interest and also which values of the sex variable are valid to add to our query.

In [ ]:
# View information about the "Sex of participant" variable for the "(SAGE)" study
found_terms_df <- bdc::extract.entries(found_terms)
found_terms_df[stringr::str_detect(found_terms_df$name, fixed("(SAGE)")), ]

The above dictionary entry shows that we can select "FEMALE", "MALE", or "NA" for gender.  For this example, let's limit our search to females.

In [ ]:
bdc::query.filter.add(query = my_query, 
                       keys = "\\NHLBI TOPMed: Study of African Americans, Asthma, Genes and Environment (SAGE) Study ( phs000921 )\\Sex of participant\\",
                       values = 'FEMALE')

In [ ]:
bdc::query.show(my_query)


#### Add continous phenotypic variable (age) to the query

Following the data dictionary search pattern just shown, we can search for the SAGE study variables related to `Subject Age`.

In [ ]:
# View information about the "subject age" variable
found_terms <- bdc::find.in.dictionary(resource = resource,
                                        term = "Subject Age")
found_terms_df <- bdc::extract.entries(found_terms)
found_terms_df[stringr::str_detect(found_terms_df$name, "(SAGE)"), ]

The dictionary entry in the output above shows the age range of data available for `Subject Age`.  

For this example, let's limit our search to a minimum age of 8 and maximum age of 35.

In [ ]:
bdc::query.filter.add(query = my_query,
                       keys = "\\NHLBI TOPMed: Study of African Americans, Asthma, Genes and Environment (SAGE) Study ( phs000921 )\\Subject age\\",
                       min = 8,
                       max = 35)

#### List available genotypic variables
To start adding genomic filters to our query, we first need to understand which genomic variables exist.

In [ ]:
# extract the genotypic vars for SAGE
geno_vars <- filter(fullVariablesDict, HpdsDataType == "info")

# display genotypic vars
head(geno_vars)

As shown in the output above, some genomic variables that can be used in queries include `Gene_with_variant`, `Variant_class`, and `Variant_severity`.

#### Add genotypic variable (Gene_with_variant) to the query
Let's use `Gene_with_variant` to view a list of genes and get more information about this variable.

In [ ]:
# View information about "Gene_with_variant" variable
found_terms <- bdc::find.in.dictionary(resource = resource,
                                        term = "Gene_with_variant")
bdc::extract.entries(found_terms)

The output shown above provides a list of values that can be used for this variable, in this case genes affected by a variant. Let's narrow our query to include the CHD8 gene.



In [ ]:
# Look for entries with variants in the CHD8 gene 
bdc::query.filter.add(query = my_query,
                       keys = "Gene_with_variant",
                       values = "CHD8")

Now that all query criteria have been entered into the query instance, we can view it by using the following line of code:

In [ ]:
# Now we show the query as it is specified
bdc::query.show(query = my_query)


Next we will take this query and retreve the data for patients with matching criteria.

## Retrieving data from the query

#### Getting query count

We have now built a query called `my_query` which contains the search criteria we are interested in.

Next, we will run a count query to find the number of matching participants.

Finally, we will run a data query to download the data.

In [ ]:
my_query_count <- bdc::query.run(query = my_query,
                                  result.type = "count")
print(my_query_count)

#### Getting query data

Once our query object is finally built, we set `result.type = "dataframe"` to retrieve the data corresponding to our query.

In [ ]:
my_query_df <- bdc::query.run(query = my_query,
                               result.type = "dataframe")

In [ ]:
dim(my_query_df)

In [ ]:
head(my_query_df, n=5)

# Data analysis example: *SERPINA1* gene and COPD

In this example, we will create a query to explore the relationship between the COPD phenotype and variants in the SERPINA1 gene. Variations of the SERPINA1 gene have been found to be a strong risk factor for COPD, which you can read more about [here](https://pubmed.ncbi.nlm.nih.gov/31661293/).

To explore this relationship, we will narrow the cohort down to participants that meet the following criteria:
* participated in the COPDgene study
* have had COPD
* have a *SERPINA1* gene variant with high or moderate severity

#### Initialize the query
Let's start by creating a new query and finding the variables pertaining to the COPDgene study using a dictionary.

In [ ]:
copd_query <- bdc::new.query(resource=resource)
copd_dictionary <- bdc::find.in.dictionary(resource = resource,
                                           term = "COPDGene") %>% bdc::extract.entries()
copdDict <- get_multiIndex_variablesDict(copd_dictionary)

**Add phenotypic variable (COPD: have you ever had COPD) to the query**

Next we will find the full variable name for "COPD: have you ever had COPD" using the `simplified_name` column and filter to this data.

In [ ]:
mask_copd <- copdDict['simplified_name'] == 'COPD: have you ever had COPD'
copd_varname <- copdDict[mask_copd, 'name'] %>%
    unlist() %>%
    unname()
copd_varname <- as.character(copd_varname)
bdc::query.filter.add(query=copd_query, keys=copd_varname, value='Yes')

**Add genomic variable (Gene_with_variant) to the query**

To add the genomic filter, we can use a dictionary to find the variable `Gene_with_variant` and filter to the `SERPINA1` gene.

In [ ]:
copd_dictionary <- bdc::new.query(resource=resource)
gene_dictionary <- bdc::find.in.dictionary(resource=resource,
                                           term="Gene_with_variant") %>% bdc::extract.entries()
gene_varname <- gene_dictionary$name
bdc::query.filter.add(query=copd_query, keys=gene_varname, value='SERPINA1')

#### Add genomic variable (Variant_severity) to the query
Finally, we can filter our results to include only variants of the *SERPINA1* gene with high or moderate severity.

In [ ]:
severity_dictionary <- bdc::find.in.dictionary(resource=resource,
                                              term = 'Variant_severity') %>% bdc::extract.entries()
severity_varname <- severity_dictionary$name
bdc::query.filter.add(query=copd_query, keys=severity_varname, value=list("HIGH", "MODERATE"))

#### Retrieve data from the query

Now that the filtering is complete, we can use this final query to get counts and perform analysis on the data.

In [ ]:
copd_results <- bdc::query.run(copd_query, result.type='dataframe')

In [ ]:
dim(copd_results)

In [ ]:
head(copd_results)